<a href="https://colab.research.google.com/github/tannerhonnef/adleotwh/blob/main/assignment4_twh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Applying Deep Learning to Earth Observation - Assignment 4**

## Instructions

Work through the assignment 2 notebook, and use this notebook to provide your answers. Be aware that the code cell for `customDataset` in this template is with complete instruction. 

To submit the assignment, you will need to use GitHub and the existing private repository you already created called `adleoxyz` (xyz is your initials)

Once you have completed the assignment:
- Commit your notebook from colab to your private GitHub repo
- The notebook should be named assignment4_xyz.ipynb, with xyz again replaced by your initials.
- There are 50 points in this assignment, with an additional 5 extra credit. 

# **Theoretical questions**

## **Answer to Q1** 
a) 

The disadvantage of using a larger kernel size is the increased number of parameters associated with the model which will also require additional computational power.

By making the network deeper by adding more convolutional layers, the model will become more complex and will be more prone to overfitting.

By pooling or performing convolution with a stride greater than 1, it is likely that there will be a loss of data because the fine details may not be reflected after the operation had been performed.

Dilated convolutions will lead to the same problems as pooling or convolutions in that the fine details will likely be lost by performing it. 



b)

One solution to using dilated convolution layers is to carefully select the rate of the dilated convolution in order to keep as much of the fine detail in the imagery as possible.

## **Answer to Q2**
a) 

ERF1 = 3+(3-1)x(1-1) = 3

ERF2 = 3+(3+(3-1))x(1-1) = 3

ERF3 = 3+(3+(3-1))x(1-1) = 3

b)

ERF1 = 3+(3-1)x(2-1) = 5

ERF2 = 5+(3+(3-1))x(2-1) = 10

ERF3 = 10+(3+(3-1))x(2-1) = 15

c)

ERF1 = 3+(3-1)x(2-1) = 5

ERF2 = 5+(3+(3-1))x(4-1) = 14

ERF3 = 14+(3+(3-1))x(8-1) = 31

## **Answer to Q3**
Parallel and dilated convolutions both increase the receptive field of the model while preserving the resolution of the model.  When arranged serially and the dilation rate is fixed among consecutive layers, resolution will be preserved and the receptive field will increase in a linear manner.  Parallel connections create a multi branch network with outputs at different spatial resolutions.  These outputs are fused together and are able to increase the receptive field and preserve spatial resolution.

# **Continue with our pipeline implementation**

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%%capture
!pip install rasterio

In [4]:
import os
import math
from pathlib import Path
from datetime import datetime, timedelta
import tqdm
import pandas as pd
import numpy as np
import cv2
import random
import rasterio
import time
import csv
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.tensorboard import SummaryWriter

from IPython.core.debugger import set_trace
from IPython.display import Image

## **Pre-process the input dataset**

Example code is provided below, which will allow you to get up and running with this assignment. However, you will learn best if you use the code you developed/modified from previous assignments to do the work, as you will start to see how it all fits together. 

In [5]:
#@title Add code for input normalization or use the existing one

def min_max_normalize_image(image, dtype=np.float32):
    """
    image_path(str) : Absolute path to the image patch.
    dtype (numpy datatype) : data type of the normalized image default is "np.float32".
    """

    # Calculate the minimum and maximum values for each band
    min_values = np.nanmin(image, axis=(1, 2))[:, np.newaxis, np.newaxis]
    max_values = np.nanmax(image, axis=(1, 2))[:, np.newaxis, np.newaxis]

    # Normalize the image data to the range [0, 1]
    normalized_img = (image - min_values) / (max_values - min_values)

    # Return the normalized image data
    return normalized_img

In [6]:
#@title Add the image augmentations functions of your choice here, or use the code below

def flip_image_and_label(image, label, flip_type):
    """
    Applies horizontal or vertical flip augmentation to an image patch and label

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        flip_type (string) : Based on the direction of flip. Can be either 
            'hflip' or 'vflip'.

    Returns:
        A tuple containing the flipped image patch and label as numpy arrays.
    """
    if flip_type == 'hflip':
        # Apply horizontal flip augmentation to the image patch
        flipped_image = cv2.flip(image, 1)
        
        # Apply horizontal flip augmentation to the label
        flipped_label = cv2.flip(label, 1)
        
    elif flip_type == 'vflip':
        # Apply vertical flip augmentation to the image patch
        flipped_image = cv2.flip(image, 0)
        
        # Apply vertical flip augmentation to the label
        flipped_label = cv2.flip(label, 0)
        
    else:
        raise ValueError("Flip direction must be 'horizontal' or 'vertical'.")
        
    # Return the flipped image patch and label as a tuple
    return flipped_image.copy(), flipped_label.copy()


def rotate_image_and_label(image, label, angle):
    """
    Applies rotation augmentation to an image patch and label.

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        angle (lost of floats) : If the list has exactly two elements they will
            be considered the lower and upper bounds for the rotation angle 
            (in degrees) respectively. If number of elements are bigger than 2, 
            then one value is chosen randomly as the roatation angle.

    Returns:
        A tuple containing the rotated image patch and label as numpy arrays.
    """
    if isinstance(angle, tuple) or isinstance(angle, list):
        if len(angle) == 2:
            rotation_degree = random.uniform(angle[0], angle[1])
        elif len(angle) > 2:
            rotation_degree = random.choice(angle)
        else:
            raise ValueError("Parameter degree needs at least two elements.")
    else:
        raise ValueError(
            "Rotation bound param for augmentation must be a tuple or list."
        )
    
    # Define the center of the image patch
    center = tuple(np.array(label.shape)/2.0)

    # Define the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_degree, 1.0)

    # Apply rotation augmentation to the image patch
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[:2], 
                                   flags=cv2.INTER_LINEAR)

    # Apply rotation augmentation to the label
    rotated_label = cv2.warpAffine(label, rotation_matrix, label.shape[:2], 
                                   flags=cv2.INTER_NEAREST)

    # Return the rotated image patch and label as a tuple
    return rotated_image.copy(), np.rint(rotated_label.copy())

For assignment 4, you are working with a dataset called "PondDataset" which consists pairs of already chipped image and labels of size: `256x256` and pixel values are already in the range of `[0, 1]`. 

**Structure:**

![PondDataset](https://drive.google.com/uc?export=view&id=12Nuy_mVXSAQpnfmpOdycnLBxeEOajMSe)

You can find the dataset in the shared drive, which is [here](https://drive.google.com/drive/folders/1hJKRa1tNQmglErELsIEk8hXEykJadmKh?usp=share_link). Please download the entire "PondDataset" folder and place it in a convenient locations in your own Google Drive 

Now you have to adapt your `ActiveLoadingDataset` from reading a "csv file", to walk through the folder structure and grab all the "tiff" files for "image" and "label" folders.



## Coding Assignment Part 1



As stated above, you can adapt the code below, or you can use your own loader and adapt it as needed for this assignment. In this case, you need to modify the loader so that it can read chips from a directory, rather than just reading a CSV.  



In [7]:
#@title Add custom dataset from previous assignment and modify to fit the requirements of assignment 4

class ActiveLoadingDataset(Dataset):
    def __init__(self, src_dir, dataset_name, usage, apply_normalization=False, 
                 transform=None, **kargs):
        r"""
        src_dir (str or path): Root of resource directory.
        dataset_name (str): Name of the training/validation dataset containing 
                              structured folders for image, label
        usage (str): Either 'train' or 'validation'.
        transform (list): Each element is string name of the transformation to  
           be used.
        """
        self.src_dir = src_dir
        self.dataset_name = dataset_name
        self.apply_normalization = apply_normalization
        self.transform = transform
        
        self.usage = usage
        assert self.usage in ["train", "validation"], "Usage is not recognized."
          
        ###### Add your code here. ######
        # hint: you need two lines of code or less: 
        #       2- make a list of filenames for the files with the ".tif" 
        #          extension using os.walk and preferably list comprehension
        #       3- sort the list of tif files.

        img_dir = Path(src_dir) / self.usage / "images"
        img_filenames = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.tif')]
        img_filenames.sort()
        lbl_dir = Path(src_dir) / self.usage / "labels"
        lbl_filenames = [os.path.join(lbl_dir, f) for f in os.listdir(lbl_dir) if f.endswith('.tif')]
        lbl_filenames.sort()
        
        ###### Add your code here. ######
        # Do the same for labels.

        self.img_chips = []
        self.lbl_chips = []
        
        ##### Add your code here #####
        # Hint: iterate through the list of string paths from the image and 
        # label lists, load the data, transpose image to have channels as the 
        # last dimension and add them to 'img_chips' and 'lbl_chips' containers.
        # iterate through the images

        

        for img_file, lbl_file in zip(img_filenames, lbl_filenames):
            with rasterio.open(img_file) as img:
                img_data = np.transpose(img.read(), (1, 2, 0))
            with rasterio.open(lbl_file) as lbl:
                lbl_data = (lbl.read(1))
            self.img_chips.append(img_data)
            self.lbl_chips.append(lbl_data)

        print('--------{} patches cropped--------'.format(len(self.img_chips)))
        

    def __getitem__(self, index):

        image_chip = self.img_chips[index]
        label_chip = self.lbl_chips[index]

        # Change the code if you are using a different augmentation
        if self.usage == "train" and self.transform:
            
            trans_flip_ls = [m for m in self.transform if "flip" in m]
            if random.randint(0, 1) and len(trans_flip_ls) > 1:
                trans_flip = random.sample(trans_flip_ls, 1)[0]
                image_chip, label_chip = flip_image_and_label(
                    image_chip, label_chip, trans_flip
                )
            
            if random.randint(0, 1) and "rotate" in self.transform:
                image_chip, label_chip = rotate_image_and_label(
                    image_chip, label_chip, angle=[0,90]
                )

        # Convert numpy arrays to torch tensors.
        # Image chips should be in 'CHW' order, if not transpose to correct 
        # order of dimensions.
        ##### Add code here #####
        image_tensor = torch.from_numpy(image_chip.transpose((2, 0, 1))).float()
        label_tensor = torch.from_numpy(np.ascontiguousarray(label_chip)).long()
        # create the two tensors here for the output below, using the correct 
        # reshaping functions

        return image_tensor, label_tensor

    def __len__(self):

        return len(self.img_chips)

**Loading your data**

In [8]:
src_dir = "/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/PondDataset"
dataset_name = "PondDataset"

transform = ["hflip", "vflip", "rotate"]

In [9]:
train_dataset = ActiveLoadingDataset(src_dir, dataset_name, usage="train", 
                                     apply_normalization=False, 
                                     transform=transform)

--------359 patches cropped--------


In [10]:
train_loader = DataLoader(train_dataset,
                          batch_size = 16, 
                          shuffle = True)

In [11]:
validation_dataset = ActiveLoadingDataset(src_dir, dataset_name, 
                                          usage="validation", 
                                          apply_normalization=False)

--------45 patches cropped--------


In [12]:
val_loader = DataLoader(validation_dataset, batch_size = 1, shuffle = False)

## **Model architecture**

For your reference I have added a complete Unet architecture.

In [13]:
#@title Add the Unet model you have designed from assignment 3 or use the existing one.


class ConvBlock(nn.Module):
    r"""This module creates a user-defined number of conv+BN+ReLU layers.
    Args:
        in_channels (int)-- number of input features.
        out_channels (int) -- number of output features.
        kernel_size (int) -- Size of convolution kernel.
        stride (int) -- decides how jumpy kernel moves along the spatial 
            dimensions.
        padding (int) -- how much the input should be padded on the borders with 
            zero.
        dilation (int) -- dilation ratio for enlarging the receptive field.
        num_conv_layers (int) -- Number of conv+BN+ReLU layers in the block.
        drop_rate (float) -- dropout rate at the end of the block.
    """

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, num_conv_layers=2, drop_rate=0):
        super(ConvBlock, self).__init__()

        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                            stride=stride, padding=padding, dilation=dilation, 
                            bias=False),
                  nn.BatchNorm2d(out_channels),
                  nn.ReLU(inplace=True), ]

        if num_conv_layers > 1:
            if drop_rate > 0:
                layers += [
                    nn.Conv2d(out_channels, out_channels, 
                              kernel_size=kernel_size, stride=stride, 
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
                    nn.Dropout(drop_rate), 
                ] * (num_conv_layers - 1)
            else:
                layers += [
                    nn.Conv2d(out_channels, out_channels, 
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True), 
                ] * (num_conv_layers - 1)

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        outputs = self.block(inputs)
        return outputs

###########################################################################

class UpconvBlock(nn.Module):
    r"""
    Decoder layer decodes the features along the expansive path.
    Args:
        in_channels (int) -- number of input features.
        out_channels (int) -- number of output features.
        upmode (str) -- Upsampling type. If "fixed" then a linear upsampling with scale factor
                        of two will be applied using bi-linear as interpolation method.
                        If deconv_1 is chosen then a non-overlapping transposed convolution will
                        be applied to upsample the feature maps. If deconv_1 is chosen then an
                        overlapping transposed convolution will be applied to upsample the feature maps.
    """

    def __init__(self, in_channels, out_channels, upmode="deconv_1"):
        super(UpconvBlock, self).__init__()

        if upmode == "fixed":
            layers = [nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True), ]
            layers += [nn.BatchNorm2d(in_channels),
                       nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False), ]

        elif upmode == "deconv_1":
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2, padding=0, dilation=1), ]

        elif upmode == "deconv_2":
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, dilation=1), ]

        # Dense Upscaling Convolution
        elif upmode == "DUC":
            up_factor = 2
            upsample_dim = (up_factor ** 2) * out_channels
            layers = [nn.Conv2d(in_channels, upsample_dim, kernel_size=3, padding=1),
                      nn.BatchNorm2d(upsample_dim),
                      nn.ReLU(inplace=True),
                      nn.PixelShuffle(up_factor), ]

        else:
            raise ValueError("Provided upsampling mode is not recognized.")

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.block(inputs)


###########################################################################

class Unet(nn.Module):
    def __init__(self, n_classes, in_channels, filter_config=None, dropout_rate=0):
        super(Unet, self).__init__()

        self.in_channels = in_channels

        if not filter_config:
            filter_config = (64, 128, 256, 512, 1024, 2048)

        assert len(filter_config) == 6

        # Contraction Path
        self.encoder_1 = ConvBlock(self.in_channels, filter_config[0], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 64x256x256
        self.encoder_2 = ConvBlock(filter_config[0], filter_config[1], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 128x128x128
        self.encoder_3 = ConvBlock(filter_config[1], filter_config[2], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 256x64x64
        self.encoder_4 = ConvBlock(filter_config[2], filter_config[3], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 512x32x32
        self.encoder_5 = ConvBlock(filter_config[3], filter_config[4], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 1024x16x16
        self.encoder_6 = ConvBlock(filter_config[4], filter_config[5], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 2048x8x8
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Expansion Path
        self.decoder_1 = UpconvBlock(filter_config[5], filter_config[4], upmode="deconv_2")  # 1024x16x16
        self.conv1 = ConvBlock(filter_config[4] * 2, filter_config[4], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_2 = UpconvBlock(filter_config[4], filter_config[3], upmode="deconv_2")  # 512x32x32
        self.conv2 = ConvBlock(filter_config[4], filter_config[3], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_3 = UpconvBlock(filter_config[3], filter_config[2], upmode="deconv_2")  # 256x64x64
        self.conv3 = ConvBlock(filter_config[3], filter_config[2], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_4 = UpconvBlock(filter_config[2], filter_config[1], upmode="deconv_2")  # 128x128x128
        self.conv4 = ConvBlock(filter_config[2], filter_config[1], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_5 = UpconvBlock(filter_config[1], filter_config[0], upmode="deconv_2")  # 64x256x256
        self.conv5 = ConvBlock(filter_config[1], filter_config[0], num_conv_layers=2, drop_rate=dropout_rate)

        self.classifier = nn.Conv2d(filter_config[0], n_classes, kernel_size=1, stride=1, padding=0)  # classNumx256x256

    def forward(self, inputs):
        # set_trace()
        e1 = self.encoder_1(inputs)  # batch size x 64 x 256 x 256
        p1 = self.pool(e1)  # batch size x 64 x 128 x 128

        e2 = self.encoder_2(p1)  # batch size x 128 x 128 x 128
        p2 = self.pool(e2)  # batch size x 128 x 64 x 64

        e3 = self.encoder_3(p2)  # batch size x 256 x 64 x 64
        p3 = self.pool(e3)  # batch size x 256 x 32 x 32

        e4 = self.encoder_4(p3)  # batch size x 512 x 32 x 32
        p4 = self.pool(e4)  # batch size x 1024 x 16 x 16

        e5 = self.encoder_5(p4)  # batch size x 1024 x 16 x 16
        p5 = self.pool(e5)  # batch size x 1024 x 8 x 8

        e6 = self.encoder_6(p5)  # batch size x 2048 x 8 x 8

        d6 = self.decoder_1(e6)  # batch size x 1024 x 16 x 16

        
        skip1 = torch.cat((e5, d6), dim=1)  # batch size x 2048 x 16 x 16

        d6_proper = self.conv1(skip1)  # batch size x 1024 x 16 x 16

        d5 = self.decoder_2(d6_proper)  # batch size x 512 x 32 x 32

        skip2 = torch.cat((e4, d5), dim=1)  # batch size x 1024 x 32 x 32

        d5_proper = self.conv2(skip2)  # batch size x 512 x 32 x 32

        d4 = self.decoder_3(d5_proper)  # batch size x 256 x 64 x 64

        skip3 = torch.cat((e3, d4), dim=1)  # batch size x 512 x 64 x 64

        d4_proper = self.conv3(skip3)  # batch size x 256 x 64 x 64

        d3 = self.decoder_4(d4_proper)  # batch size x 128 x 128 x 128

        skip4 = torch.cat((e2, d3), dim=1)  # batch size x 256 x 128 x 128

        d3_proper = self.conv4(skip4)  # batch size x 128 x 128 x 128

        d2 = self.decoder_5(d3_proper)  # batch size x 64 x 256 x 256

        skip5 = torch.cat((e1, d2), dim=1)  # batch size x 128 x 256 x 256

        d2_proper = self.conv5(skip5)  # batch size x 64 x 256 x 256

        d1 = self.classifier(d2_proper)  # batch size x classNum x 256 x 256

        return d1

**Initializing your model**

In [14]:
n_classes = 2
in_channels = 6
filter_config = (32, 64, 128, 256, 512, 1024)
dropout_rate = 0.15

In [15]:
model = Unet(n_classes, in_channels, filter_config, dropout_rate)

## **Customized loss function**

You will want to add two here, which you can copy from the main assignment notebook. 

In [16]:
#@title Add your choices of loss function here 
class BalancedCrossEntropyLoss(nn.Module):
    '''
    Balanced cross entropy loss by weighting of inverse class ratio
    Params:
        ignore_index (int): Class index to ignore
        reduction (str): Reduction method to apply to loss, return mean over batch if 'mean',
            return sum if 'sum', return a tensor of shape [N,] if 'none'
    Returns:
        Loss tensor according to arg reduction
    '''

    def __init__(self, ignore_index=-100, reduction='mean'):
        super(BalancedCrossEntropyLoss, self).__init__()
        self.ignore_index = ignore_index
        self.reduction = reduction

    def forward(self, predict, target):
        # get class weights
        class_counts = torch.bincount(target.view(-1), minlength=predict.shape[1])
        class_weights = 1.0 / torch.sqrt(class_counts.float())

        # set weight of ignore index to 0
        if self.ignore_index >= 0 and self.ignore_index < len(class_weights):
            class_weights[self.ignore_index] = 0

        # normalize weights
        class_weights /= torch.sum(class_weights)

        # apply class weights to loss function
        loss_fn = nn.CrossEntropyLoss(weight=class_weights, ignore_index=self.ignore_index, reduction=self.reduction)

        return loss_fn(predict, target)

In [17]:
class BinaryDiceLoss(nn.Module):
    '''
        Dice loss of binary class
        Params:
            smooth (float): A float number to smooth loss, and avoid NaN error, default: 1
            p (int): Denominator value: \sum{x^p} + \sum{y^p}, default: 2. Used
                     to control the sensitivity of the loss.
            predict (torch.tensor): Predicted tensor of shape [N, *]
            target (torch.tensor): Target tensor of same shape with predict
        Returns:
            Loss tensor
    '''
    def __init__(self, smooth=1, p=1):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p

    def forward(self, predict, target):

        assert predict.shape == target.shape, "predict & target shape do not match"
        predict = predict.contiguous().view(-1)
        target = target.contiguous().view(-1)

        num = 2 * (predict * target).sum() + self.smooth
        den = (predict.pow(self.p) + target.pow(self.p)).sum() + self.smooth
        loss = 1 - num / den

        return loss


class DiceLoss(nn.Module):
    '''
        Dice loss
        Params:
            weight (torch.tensor): Weight array of shape [num_classes,]
            ignore_index (int): Class index to ignore
            predict (torch.tensor): Predicted tensor of shape [N, C, *]
            target (torch.tensor): Target tensor either in shape [N,*] or of same shape with predict
            reduction (str): Reduction method.
        Returns:
            same as BinaryDiceLoss
    '''
    def __init__(self, weight=None, ignore_index=-100, smooth=1, p=1, reduction='sum'):
        super(DiceLoss, self).__init__()
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dice = BinaryDiceLoss(smooth, p)
        if weight is not None:
            self.weight = weight.cuda()
        else:
            self.weight = None

    def forward(self, predict, target):
        #set_trace()
        if predict.shape == target.shape:
            pass
        elif len(predict.shape) == 4:
            target = F.one_hot(target, num_classes=predict.shape[1]).permute(0, 3, 1, 2).contiguous()
        else:
            assert 'predict shape not applicable'

        self.weight = torch.Tensor([1. / predict.shape[1]] * predict.shape[1]).cuda() if self.weight is None else self.weight
        predict = F.softmax(predict, dim=1)

        if self.ignore_index >= 0:
            target = torch.where(target == self.ignore_index, torch.zeros_like(target), target)
            predict = torch.where(target == self.ignore_index, torch.zeros_like(predict), predict)

        total_loss = 0
        for i in range(predict.shape[1]):
            dice_loss = self.dice(predict[:, i], target[:, i])

            assert self.weight.shape[0] == predict.shape[1], \
                    'Expect weight shape [{}], get[{}]'.format(predict.shape[1], self.weight.shape[0])
            dice_loss *= self.weight[i]
            total_loss += dice_loss

        if self.reduction == 'mean':
            loss = total_loss / predict.shape[1]
        elif self.reduction == 'sum':
            loss = total_loss

        return loss

## **Coding assignment part 2**: training the network



In the sections below you need to complete the three functions that you need to train and validate the network over a specified number of epochs  

**HOW To:**

Properly use the "criterion" argument inside the "train" and "validation" functions:
Pass the argument to the function as a string with `()` like: "BalancedCrossEntropyLoss()"

Now as you try to use the argument inside both "train" and "validation" functions, use `eval()` like: 

`loss = eval(criterion)(tensor A, tensor B)`

**Note:**

`eval()` is a built-in Python function that allows you to evaluate a string expression as a Python code. It takes a string as an argument and evaluates the expression contained in it. The result of the evaluation is then returned.

In [18]:
#@title Complete the function to optimize over a batch of training images and labels
def train(trainData, model, optimizer, criterion, device=True, train_loss=[]):
    """
        Train the model using provided training dataset.
        Params:
            trainData (DataLoader object) -- Batches of image chips from PyTorch 
                custom dataset (AquacultureData).
            model -- Choice of segmentation model.
            optimizer -- Chosen optimization algorithm to update model parameters.
            criterion -- Chosen function to calculate loss over training samples.
            device (bool, optional) -- Decide whether to use GPU, default is True.
            train_loss (empty list, optional) -- ???????????????????????????
    """

    model.train()

    # Mini batch iteration
    train_epoch_loss = 0
    train_loss=[]
    train_batches = len(trainData)

    for img_chips, labels in trainData:

        #Add code to put image and label on the 'device'.
        # one line for each.
        img_chips = img_chips.to(device)
        label = labels.to(device)
        # Add code to clear the 'optimizer' from existing gradients (1 line)
        optimizer.zero_grad()
        # Pass image through the model to obtain prediction (1 line)
        pred = model(img_chips)
        # calculate loss based on 'model prediction' and label (1 line)
        loss = eval(criterion)(pred, label)
        # Add current loss (loss.item()) to 'train_epoch_loss' counter (1 line)
        train_epoch_loss += loss.item()
        # do the backward pass to calculate gradients with respect to the loss (1 line)
        loss.backward()
        # update model weights by invoking the proper method on 'optimizer'
        optimizer.step()

    train_loss.append(train_epoch_loss / train_batches)
    print('Training loss: {:.4f}'.format(train_epoch_loss / train_batches))

Besides training the network, it's important to evaluate its performance on a separate "validation dataset" to ensure that it's not overfitting to the training data. The validation process is similar to the training process, but the network is set to evaluation mode using `model.eval()` and the gradients are not computed.

In [19]:
#@title Complete the function to iterate validation images and labels


def validate(valData, model, criterion, device, val_loss=[]):
    """
        Evaluate the model on separate Landsat scenes.
        Params:
            valData (DataLoader object) -- Batches of image chips from PyTorch custom dataset(AquacultureData)
            model -- Choice of segmentation Model.
            criterion -- Chosen function to calculate loss over validation samples.
            buffer: Buffer added to the targeted grid when creating dataset. This allows loss to calculate
                at non-buffered region.
            gpu (binary,optional): Decide whether to use GPU, default is True
            valLoss (empty list): To record average loss for each epoch
    """

    model.eval()

    # mini batch iteration
    eval_epoch_loss = 0
    eval_loss=[]

    for img_chips, labels in valData:

        img = Variable(img_chips, requires_grad=False)
        label = Variable(labels, requires_grad=False)

        #Add code to put image and label on the 'device'.
        # one line for each.
        image = img.to(device)
        label = label.to(device)

        # Pass image through the model to obtain prediction (1 line)
        pred = model(image)
        # calculate loss based on 'model prediction' and label (1 line)
        loss = eval(criterion)(pred, label)
        # Add current loss (loss.item()) to 'train_epoch_loss' counter (1 line)
        eval_epoch_loss += loss.item()

    print('validation loss: {}'.format(eval_epoch_loss / len(valData)))

    if val_loss != None:
        val_loss.append(float(eval_epoch_loss / len(valData)))


In [20]:
#@title Complete the function to iterate over the desired number of epochs

def epochIterater(trainData, valData, model, criterion, WorkingFolder, initial_lr, num_epochs):
    r"""
    Epoch iteration for train and evaluation.
    
    Arguments:
    trainData (dataloader object): Batch grouped data to train the model.
    evalData (dataloader object): Batch grouped data to evaluate the model.
    model (pytorch.nn.module object): initialized model.
    initial_lr(float): The initial learning rate.
    num_epochs (int): User-defined number of epochs to run the model.
    
    """

    train_loss = []
    val_loss = []

    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if device.type == "cuda":
        print('----------GPU available----------')
        gpu = True
        model = model.to(device)
    else:
        print('----------No GPU available, using CPU instead----------')
        gpu = False
        model = model
    
    writer = SummaryWriter(WorkingFolder)
    optimizer = optim.Adam(model.parameters(),
                           lr=initial_lr,
                           betas=(0.9, 0.999),
                           eps=1e-08,
                           weight_decay=5e-4,
                           amsgrad=False)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=3,
                                          gamma=0.98)

    # Add your code here
    for t in range(num_epochs):
    # you need to loop through the epochs and perform the following:
    # print the current epoch number out of the total epochs (e.g. "epoch: 2/10")(1 line)
      print(f"Epoch [{t+1}/{num_epochs}]")
    # start the timer (1 line)
      start_time = datetime.now()
    # do model fit on the training data for single epoch (1 line)
      train(trainData, model, optimizer, criterion, device, train_loss=train_loss)
    # do model validation on the validation dataset for one epoch (1 line)
      validate(valData, model, criterion, device, val_loss=val_loss)
    # take a step to update the 'scheduler'. (1 line)
      scheduler.step()
    # Print the updated learning rate.
      print(scheduler.get_last_lr())
    # use "add_scalars" method with your writer to save the train and validation loss to graph
      writer.add_scalar('Loss/train', np.random.random(), t)
      writer.add_scalar('Loss/test', np.random.random(), t)
      writer.add_scalar('Accuracy/train', np.random.random(), t)
      writer.add_scalar('Accuracy/test', np.random.random(), t)

    # using tensorboard package later. 
    
    writer.close()

    duration_in_sec = (datetime.now() - start_time).seconds
    duration_format = str(timedelta(seconds=duration_in_sec))
    print("--------------- Training finished in {} ---------------".format(duration_format))

### Demonstrate the code
Run the model training and validation for a specified number of epochs (e.g. 15), and then save the results. Train / validate twice, once using your first loss function, and again using your second loss function.  





In [21]:
num_epochs = 10
initial_lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=initial_lr)
criterion = "BalancedCrossEntropyLoss()"
path ="/content/gdrive/MyDrive/clarkFiles/geog215/assignment4/working/model1"

epoch_it =  epochIterater(train_loader, val_loader, model=model, criterion=criterion,
                          WorkingFolder=path, initial_lr=initial_lr, num_epochs=num_epochs)

----------GPU available----------
Epoch [1/10]
Training loss: 0.4184
validation loss: 1.1558476832177904
[0.001]
Epoch [2/10]
Training loss: 0.2979
validation loss: 0.7719050778283013
[0.001]
Epoch [3/10]
Training loss: 0.2444
validation loss: 0.5679082920153936
[0.00098]
Epoch [4/10]
Training loss: 0.2536
validation loss: 0.4862559696038564
[0.00098]
Epoch [5/10]
Training loss: 0.2288
validation loss: 0.4129699945449829
[0.00098]
Epoch [6/10]
Training loss: 0.2050
validation loss: 0.46951961947811977
[0.0009603999999999999]
Epoch [7/10]
Training loss: 0.1986
validation loss: 0.519178647796313
[0.0009603999999999999]
Epoch [8/10]
Training loss: 0.2236
validation loss: 0.6124481298857265
[0.0009603999999999999]
Epoch [9/10]
Training loss: 0.1889
validation loss: 0.3771999110778173
[0.0009411919999999999]
Epoch [10/10]
Training loss: 0.1757
validation loss: 0.5428740438487795
[0.0009411919999999999]
--------------- Training finished in 0:00:10 ---------------


In [22]:
#@title Save model results 1 in a directory of choice in your gdrive
path = "/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/working/model1/results"
torch.save(model.state_dict(), path)

In [23]:
#@title Train/validate 2
num_epochs = 10
initial_lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=initial_lr)
criterion = "DiceLoss()"
path ="/content/gdrive/MyDrive/clarkFiles/geog215/assignment4/working/model2"

epoch_it =  epochIterater(train_loader, val_loader, model=model, criterion=criterion,
                          WorkingFolder=path, initial_lr=initial_lr, num_epochs=num_epochs)

----------GPU available----------
Epoch [1/10]
Training loss: 0.1431
validation loss: 0.3039872381422255
[0.001]
Epoch [2/10]
Training loss: 0.1221
validation loss: 0.3294672012329102
[0.001]
Epoch [3/10]
Training loss: 0.1057
validation loss: 0.2806210047668881
[0.00098]
Epoch [4/10]
Training loss: 0.1035
validation loss: 0.35491414268811544
[0.00098]
Epoch [5/10]
Training loss: 0.0993
validation loss: 0.3570214006635878
[0.00098]
Epoch [6/10]
Training loss: 0.1024
validation loss: 0.2480929818418291
[0.0009603999999999999]
Epoch [7/10]
Training loss: 0.1125
validation loss: 0.28609017928441366
[0.0009603999999999999]
Epoch [8/10]
Training loss: 0.1062
validation loss: 0.2848147670427958
[0.0009603999999999999]
Epoch [9/10]
Training loss: 0.0969
validation loss: 0.2767730540699429
[0.0009411919999999999]
Epoch [10/10]
Training loss: 0.0965
validation loss: 0.2389099074734582
[0.0009411919999999999]
--------------- Training finished in 0:00:10 ---------------


In [24]:
#@title Save model results 1 in a directory of choice in your gdrive
path = "/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/working/model2/results"
torch.save(model.state_dict(), path)


## **Evaluation and accuracy metrics**

**Note:**

If you have disconnected from the Colab session or restarted the kernel, then before doing the evaluation on the validation dataset you must initialize your model once more and load the trained weights onto your model.

In [25]:
#@title code for metric class

class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        Acc = np.nanmean(Acc)
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

## Coding Assignment Part 3

Complete the code to undertake model evaluation below. Evaluate twice: once for each model trained with a different loss function.  

In [26]:
def do_accuracy_evaluation(model, dataloader, num_classes, sv_dir):
    evaluator = Evaluator(num_classes)

    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)

            # add batch to evaluator
            evaluator.add_batch(labels.cpu().numpy(), preds.cpu().numpy())

    # calculate evaluation metrics
    pixel_accuracy = evaluator.Pixel_Accuracy()
    mean_accuracy = evaluator.Pixel_Accuracy_Class()
    mean_IoU = evaluator.Mean_Intersection_over_Union()
    frequency_weighted_IoU = evaluator.Frequency_Weighted_Intersection_over_Union()

    # save accuracies to csv
    with open(sv_dir, mode='w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Pixel Accuracy', 'Mean Accuracy', 'Mean IoU', 'Frequency Weighted IoU'])
        writer.writerow([pixel_accuracy, mean_accuracy, mean_IoU, frequency_weighted_IoU])

    return pixel_accuracy, mean_accuracy, mean_IoU, frequency_weighted_IoU

In [27]:
#@title Demonstrate evaluation of model 1
sv_dir = "/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/working/model1/accuracies.csv"
do_accuracy_evaluation(model, train_loader, num_classes=2, sv_dir=sv_dir)

(0.943795206819072, 0.8939267757098368, 0.8364519680264444, 0.895047474873476)

In [34]:
#@title Demonstrate evaluation of model 2
sv_dir = "/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/working/model2/accuracies.csv"
do_accuracy_evaluation(model, train_loader, num_classes=2, sv_dir=sv_dir)

(0.908692128811042, 0.7901484451529817, 0.7297397249170274, 0.8304577092395075)

## **Visualizing activation maps and learned kernels from intermidiate layers in the network**

In [29]:
#Load the multispectral satellite image
with rasterio.open("/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/PondDataset/train/images/LC08_L1TP_011060_20190426_20190426_01_RT_band_chip1.tif") as dataset:
    # Read the image data as a numpy array and reshape to HWC
    image = dataset.read().transpose([1, 2, 0])

# Normalize the image data to be between 0 and 1
image = min_max_normalize_image(image)

# Convert the image to a PyTorch tensor and add a batch dimension
image_tensor = torch.from_numpy(image).float().permute(2, 0, 1).unsqueeze(0)

In [30]:
# Define the CNN model and load the pre-trained weights
model = model
model.load_state_dict(torch.load('/content/gdrive/MyDrive/clarkFiles/dsci215/assignment4/working/model1/results'))

print(model)

Unet(
  (encoder_1): ConvBlock(
    (block): Sequential(
      (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Dropout(p=0.15, inplace=False)
    )
  )
  (encoder_2): ConvBlock(
    (block): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
 

In [31]:
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [32]:
# Visualize the activation map for a particular layer

layer_index = 2  # Choose the index of the layer to visualize (0-indexed)
layer_name = f'conv{layer_index + 1}'  # Layer name for displaying in the plot title
activation = {}

In [33]:
# Set the model to evaluation mode
model.eval()

model.conv1.register_forward_hook(get_activation(layer_name))

with torch.no_grad():
    model(image_tensor)

activation = activation.squeeze().numpy()

plt.imshow(activation[layer_index])
plt.title(f'Activation Map for {layer_name}')
plt.show()

RuntimeError: ignored